In [1]:
import os
from owlready2 import *
import pandas as pd
from rdflib import Graph

def convert_to_owl(input_path, output_owl_path): 
    g = Graph()
    input_format = 'ttl' if input_path.endswith('.ttl') else 'xml'
    g.parse(input_path, format=input_format)
    # Serialize the graph to RDF/XML format (commonly used for .owl files)
    g.serialize(destination=output_owl_path, format='pretty-xml')

def count_ontology_elements(ontology_path):
    try:
        
         
        onto = get_ontology(ontology_path).load()

        print(f"Processing: {ontology_path}")
        
        # Count the elements
        num_classes = len(list(onto.classes()))
        num_individuals = len(list(onto.individuals()))
        num_object_properties = len(list(onto.object_properties()))
        num_data_properties = len(list(onto.data_properties()))
        
        counts = {
            "filename": os.path.basename(ontology_path),
            "classes": num_classes,
            "individuals": num_individuals,
            "object_properties": num_object_properties,
            "data_properties": num_data_properties
        }
        print(counts)
        return counts
    
    except Exception as e:
        print(f"Error loading {ontology_path}: {e}")
        return None

def count_elements_in_directory(directory_path): 
    ontology_counts = {}
    
    # Supported ontology file extensions
    supported_extensions = (".owl", ".ttl", ".rdf")
    
    # Create a directory for converted files if it doesn't exist
    converted_dir = os.path.join(directory_path, 'converted_files_bkp')
    os.makedirs(converted_dir, exist_ok=True)
     
    for filename in os.listdir(directory_path):
        if filename.endswith(supported_extensions):
            ontology_path = os.path.join(directory_path, filename)
            
            # If the file is .ttl or .rdf, convert it to .owl and move it to 'converted_files_bkp' folder
            if filename.endswith(('.ttl', '.rdf')): 
                converted_path = os.path.join(directory_path, os.path.splitext(filename)[0] + '.owl')
                convert_to_owl(ontology_path, converted_path)
                
                # move ttl / rdf to converted_files_bkp
                new_original_path = os.path.join(converted_dir, filename)
                os.rename(ontology_path, new_original_path)
                
                 
                ontology_path = converted_path
            
            # Count classes/properties in the .owl file
            counts = count_ontology_elements(ontology_path)
            if counts:
                ontology_counts[filename] = counts
    
    return ontology_counts 

In [5]:
directory_path = "../Data/assertion_evidence/" 
!ls $directory_path

ls: ../Data/assertion_evidence/: Operation not permitted


In [3]:
ontology_counts_assertion_evidence = count_elements_in_directory(directory_path)

df_assertion_evidence = pd.DataFrame.from_dict(ontology_counts_assertion_evidence,orient='index')
df_assertion_evidence_reset = df_assertion_evidence.reset_index(drop=True)
df_assertion_evidence_reset

""


In [4]:
print(df_assertion_evidence_reset.to_string(index=False))

Empty DataFrame
Columns: []
Index: []


In [6]:
directory_path = "../Data/provenance"
ontology_counts_provenance = count_elements_in_directory(directory_path)

df_provenance = pd.DataFrame.from_dict(ontology_counts_provenance,orient='index')
df_reset_provenance = df_provenance.reset_index(drop=True)
df_reset_provenance

""


In [6]:
!pwd

.


Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x1066ee6d0>>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3920, in atexit_operations
    self._atexit_once()
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3899, in _atexit_once
    self.reset(new_session=False)
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 1399, in reset
    self.history_manager.reset(new_session)
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/history.py", line 607, in reset
    self.dir_hist[:] = [Path.cwd()]
                        ^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/pathlib.py", line 907, in cwd
    return cls(os.getcwd())
               ^^^^^^^^^^^
PermissionError: [Errno 1] Operation not permitted
